# Steps of the Notebook
<a id="toc"></a>
- [1. Template Description](#1)
- [2. Import Libraries](#2)
- [3. Pass the data path](#3)
- [4. Adjust Diminsions of the Images](#4)
- [5.Splitting the data into Training and Testing datasets](#5)
- [6. Normalization](#6)
- [7. One Hot Encoding](#7)
- [8. Adjust Early Stop](#8)
- [9. Adjust ANN and it's Parameters](#9)
    - [9.1 Plot Loss Curve ANN](#9.1)
    - [9.2 Plot Accuarcy Curve ANN](#9.2)
    - [9.3 Plot Confusion Matrix ANN](#9.3)
    - [9.4 Generate Classification Report](#9.4)
- [10. Adjust CNN and it's Parameters](#10)
    - [10.1 Plot Loss Curve CNN](#10.1)
    - [10.2 Plot Accuarcy Curve CNN](#10.2)
    - [10.3 Plot Confusion Matrix CNN](#10.3)
    - [10.4 Generate Classification Report](#10.4)

# Template Description

- **The template is good in dealing with images dataset. All you have to do is enter the path of the data and modify in some of the parameters of the neural networks (if you want to modify them) and you are done and it will do the training process, visualize loss and accuarcy curves, plot confusion matrix and generating classification Report and saves the best model**

- **The template supports images dataset only, and all images for each class must be placed in a separate folder**

- **For example, if I have two types of classification dogs and cats, you must put the pictures of cats in a folder and the pictures of dogs in another folder, then put these two folders in one folder, and then pass this folder**
- **What is the Reason for That? it will automatically determine the number of classes in the data without the need for you to write them manually**<br><br>

- **Real Example, Let's Back to the Cats and Dogs classification**<br>
- **1 - Put Cats Images in a folder Called "Cats" (Name the Directory whatever you want I'm just assuming names)**<br>
- **2 - Put Dogs Images in a folder Called "Dogs"**<br>
- **3 - Then the two folders "Cats and Dogs" put them in a folder called "Data"**<br>
- **4 - So "Data" Folder contains Cats Folder and Dogs Folder**<br>
- **5 -The Last Step is Just Path the Path of the Data Folder in the 'data_path' variable**<br>

- **it's purpose is to save hours of your time**

# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from seaborn import heatmap
import cv2
import random
from os import listdir

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import  LabelEncoder

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
from tensorflow.math import confusion_matrix
from tensorflow.keras.utils import to_categorical

# Pass the Data path

In [ ]:
data_path = ''

In [ ]:
# Setting path and creating empty lists
dir = data_path
root_dir = listdir(dir)
image_list, label_list = [], []

# Adjust Image Diminsions

In [ ]:
image_width = 128
image_height = 128
image_channel = 3

In [ ]:
# Reading and converting image to numpy array
for directory in root_dir:
    for files in listdir(f"{dir}/{directory}"):
        image_path = f"{dir}/{directory}/{files}"
        image = load_img(image_path, target_size=(image_width, image_height))
        image = img_to_array(image)
        image_list.append(image)
        label_list.append(directory)

In [ ]:
# Visualize the number of classes count
label_counts = pd.DataFrame(label_list).value_counts()
label_counts

In [ ]:
num_classes = len(label_counts)
num_classes

# Splitting the data into Training and Testing

In [ ]:
# Splitting dataset
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.3, random_state = 0) 

# Normlization

In [ ]:
# Normalize and reshape data
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, image_width, image_height, image_channel)
x_test = x_test.reshape( -1, image_width, image_height, image_channel)

# One Hot Encoding

In [ ]:
# One hot encoding on target variable
LE = LabelEncoder()
y_train = to_categorical(LE.fit_transform(y_train))
y_test = to_categorical(LE.fit_transform(y_test))

# Adjust Early Stop

In [ ]:
early_stopping = callbacks.EarlyStopping(
        patience=10,
        min_delta=0,
        monitor='val_loss',
        restore_best_weights=True,
        verbose=0,
        mode='min', 
        baseline=None,
    )
plateau = callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2, 
            patience=4, 
            verbose=0,
            mode='min')

# Adjust ANN and it's Parameters

In [ ]:
hidden_layer1 = 2048
hidden_layer2 = 1024
hidden_layer3 = 512
hidden_layer4 = 256
hidden_layer5 = 128
hidden_layers_activation = 'relu'
output_layer_activation = 'softmax'

LEARNING_RATE = 0.001
loss_function = 'categorical_crossentropy'
ANN_epochs = 1000
batch_size = 10

ANN_model_checkpoint = callbacks.ModelCheckpoint('ANN_best_model.h5', monitor='val_loss', mode='min', patience = 10 ,save_best_only=True)


In [ ]:
def load_model(): 
    model = Sequential([
        Flatten(input_shape=(image_width,image_height,image_channel)),
        Dense(hidden_layer1, activation =hidden_layers_activation),
        Dense(hidden_layer2, activation =hidden_layers_activation),
        Dense(hidden_layer3, activation =hidden_layers_activation),
        Dense(hidden_layer4, activation =hidden_layers_activation),
        Dense(hidden_layer5, activation =hidden_layers_activation),
        Dense(num_classes, activation=output_layer_activation),
    ])
    model.compile(
        optimizer= Adam(learning_rate=LEARNING_RATE),
        loss=loss_function,
        metrics=['acc'],
    )
    return model

ann_model = load_model()
history_ann = ann_model.fit(  x_train , y_train,
                validation_data = (x_test , y_test),
                epochs = ANN_epochs,
                callbacks = [early_stopping , plateau, ANN_model_checkpoint],
                batch_size = batch_size
              )

In [ ]:
ann_model.evaluate(x_test, y_test)

##  Loss Curve ANN

In [ ]:
plt.figure(figsize=(11,8))
loss_train = history_ann.history['loss']
loss_val = history_ann.history['val_loss']
epochs = range(1,len(loss_train) + 1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Accuarcy Curve ANN

In [ ]:
plt.figure(figsize=(11,8))
val_train = history_ann.history['acc']
val_acc = history_ann.history['val_acc']
epochs = range(1,len(val_acc)+1)
plt.plot(epochs, val_train, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Confusion Matrix ANN

In [ ]:
def conf_matrix(y_test, y_pred):
    y_predicted_labels = [np.argmax(i) for i in y_pred]
    y_predicted_labels = np.array(y_predicted_labels)
    y_test = np.argmax(y_test, axis=1)
    cm = confusion_matrix(labels=y_test, predictions=y_predicted_labels)
    plt.figure(figsize=(12,10))
    heatmap(cm, annot=True, fmt='d');

In [ ]:
y_pred_ann = ann_model.predict(x_test)
conf_matrix(y_test, y_pred_ann)

## Classification Report ANN

In [ ]:
def class_report(y_test, y_pred):
    y_predicted_labels = [np.argmax(i) for i in y_pred]
    y_predicted_labels = np.array(y_predicted_labels)
    y_test = np.argmax(y_test, axis=1)
    print(classification_report(y_test, y_predicted_labels))

In [ ]:
class_report(y_pred_ann, y_test)

# Adjust CNN and it's Parameters

In [ ]:
filters1 = 16
filters2 = 32
filters3 = 64
filters4 = 128

kernel_size = (3,3)
conv_activation = 'LeakyRelu'

pool_size1 = (5,5)
pool_size2 = (3,3)
pool_size3 = (2,2)

hidden_layer1 = 64
hidden_layer2 = 32
hidden_layer3 = 16

hidden_layer_activation = 'relu'
output_layer_activation = 'softmax'

LEARNING_RATE = 0.0005
loss_function = 'categorical_crossentropy'
CNN_epochs = 1000

CNN_model_checkpoint = callbacks.ModelCheckpoint('CNN_best_model.h5', monitor='val_loss', mode='min', patience = 10 ,save_best_only=True)

In [ ]:
def load_model():
    cnn_model = Sequential([
        Conv2D(filters1, kernel_size = kernel_size,input_shape=(image_width,image_height,image_channel), activation=conv_activation),
        MaxPooling2D(pool_size=pool_size1),
        Conv2D(filters2, kernel_size = kernel_size, activation=conv_activation),
        MaxPooling2D(pool_size=pool_size1),
        Conv2D(filters3, kernel_size = kernel_size, activation= conv_activation),
        MaxPooling2D(pool_size=pool_size1),
        Conv2D(filters4, kernel_size = kernel_size, activation= conv_activation),
        MaxPooling2D(pool_size=pool_size1),
        
        Flatten(),
        
        Dense(hidden_layer1, activation =hidden_layer_activation),
        Dense(hidden_layer2, activation =hidden_layer_activation),
        Dense(hidden_layer3, activation =hidden_layer_activation),
        Dense(num_classes, activation=output_layer_activation)    
    ])
    cnn_model.compile(
        Adam(learning_rate=LEARNING_RATE),
        loss = loss_function,
        metrics = ['acc']
    )
    return model



cnn_model = load_model()
history_cnn = cnn_model.fit(  x_train , y_train,
                validation_data = (x_test , y_test),
                epochs = CNN_epochs,
                callbacks = [early_stopping , plateau, CNN_model_checkpoint],
              )


In [ ]:
cnn_model.evaluate(x_test, y_test)

## Loss Curve CNN

In [ ]:
plt.figure(figsize=(11,8))
loss_train = history_cnn.history['loss']
loss_val = history_cnn.history['val_loss']
epochs = range(1,len(loss_train) + 1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Accuarcy Curve CNN

In [ ]:
plt.figure(figsize=(11,8))
acc_train = history_cnn.history['acc']
acc_val = history_cnn.history['val_acc']
epochs = range(1,len(acc_train) + 1)
plt.plot(epochs, acc_train, 'g', label='Training Accuarcy')
plt.plot(epochs, acc_val, 'b', label='validation Accuarcy')
plt.title('Training and Validation Accuarcy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Confusion Matrix CNN

In [ ]:
y_pred_cnn = cnn_model.predict(x_test)
conf_matrix(y_test, y_pred_cnn)

## Class Report CNN

In [ ]:
class_report(y_test, y_pred_cnn)